## We are using TSA checkpoint travel numbers as X dataset

In [35]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [36]:
# URL object
url = 'https://www.tsa.gov/coronavirus/passenger-throughput'
# page object
page = requests.get(url)

In [37]:
page

<Response [200]>

#### Response 200
We can collect data from this url

In [38]:
soup = BeautifulSoup(page.text, 'lxml')

In [39]:
# Extract information from tag <table>
table = soup.find('table')

In [40]:
# Extract headers from the web page
headers = []
for i in table.find_all('th'):
    title = i.text
    headers.append(title)
    
print("Columns:", headers)

Columns: ['Date', '2022', '2021', '2020', '2019']


In [41]:
passenger_travel_df = pd.DataFrame(columns = headers)

In [42]:
# Extract other information from the table
for j in table.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text for i in row_data]
    length = len(passenger_travel_df)
    passenger_travel_df.loc[length] = row

In [43]:
passenger_travel_df.shape

(364, 5)

In [44]:
passenger_travel_df.tail()

,Date,2022,2021,2020,2019
359,5/22/2021,,"1,550,044","253,190","2,124,825"
360,5/21/2021,,"1,820,433","348,673","2,792,670"
361,5/20/2021,,"1,728,496","318,449","2,673,635"
362,5/19/2021,,"1,496,089","230,367","2,472,123"
363,5/18/2021,,"1,408,017","190,477","2,312,727"


In [45]:
passenger_travel_df['Date'] = pd.to_datetime(passenger_travel_df['Date'])

### This dataset contains data from 2019 to 2022 till 5/15/2022

In [46]:
passenger_travel_df['2022'] = passenger_travel_df['2022'].str.replace(",", "").replace(" ", "")
passenger_travel_df['2021'] = passenger_travel_df['2021'].str.replace(",", "").replace(" ", "")
passenger_travel_df['2020'] = passenger_travel_df['2020'].str.replace(",", "").replace(" ", "")
passenger_travel_df['2019'] = passenger_travel_df['2019'].str.replace(",", "").replace(" ", "")

In [47]:
# passenger_travel_df.head(25)

In [48]:
# passenger_travel_df.to_csv('passenger_travel_data.csv', index=False)

In [49]:
df = pd.read_csv('passenger_travel_data.csv')
df = df.sort_values(by=['Date'])
df['date_old'] = df['Date']
# df

In [50]:
def get_values(row):
#     print(row)
    if row['year'] == '2022':
        value = row['2022']
    elif row['year'] == '2021':
        value = row['2021']
    elif row['year'] == '2020':
        value = row['2020']
    elif row['year'] == '2019':
        value = row['2019']   
    return value

In [51]:
df1 = df.copy()
df1['Date'] = pd.to_datetime(df1['Date'])
df1['Date'] = pd.to_datetime(df1['Date'].dt.strftime('2022-%m-%d'))
df1['year'] = df1['Date'].dt.strftime('%Y')
df1['travel_count'] = df1.apply(lambda row: get_values(row), axis=1)
df1.drop(['date_old', '2022', '2021', '2020', '2019', 'year'], axis=1, inplace=True)
df1 = df1.sort_values(by=['Date'])
# df1

In [52]:
df2 = df.copy()
df2['Date'] = pd.to_datetime(df2['Date'])
df2['Date'] = pd.to_datetime(df2['Date'].dt.strftime('2021-%m-%d'))
df2['year'] = df2['Date'].dt.strftime('%Y')
df2['travel_count'] = df2.apply(lambda row: get_values(row), axis=1)
df2.drop(['date_old', '2022', '2021', '2020', '2019', 'year'], axis=1, inplace=True)
df2 = df2.sort_values(by=['Date'])
# df2

In [53]:
df3 = df.copy()
df3['Date'] = pd.to_datetime(df3['Date'])
df3['Date'] = pd.to_datetime(df3['Date'].dt.strftime('2020-%m-%d'))
df3['year'] = df3['Date'].dt.strftime('%Y')
df3['travel_count'] = df3.apply(lambda row: get_values(row), axis=1)
df3.drop(['date_old', '2022', '2021', '2020', '2019', 'year'], axis=1, inplace=True)
df3 = df3.sort_values(by=['Date'])
# df3

In [54]:
df4 = df.copy()
df4['Date'] = pd.to_datetime(df4['Date'])
df4['Date'] = pd.to_datetime(df4['Date'].dt.strftime('2019-%m-%d'))
df4['year'] = df4['Date'].dt.strftime('%Y')
df4['travel_count'] = df4.apply(lambda row: get_values(row), axis=1)
df4.drop(['date_old', '2022', '2021', '2020', '2019', 'year'], axis=1, inplace=True)
df4 = df4.sort_values(by=['Date'])
# df4

### Combining all the years to get two columns: Date and Travel count

In [55]:
df_combined = df4.append([df3, df2, df1], ignore_index=True)
df_combined.set_index('Date', inplace=True)
df_combined.index = pd.to_datetime(df_combined.index)
df_combined

,travel_count
Date,
2019-01-01,2345103.0
2019-01-02,2202111.0
2019-01-03,2150571.0
2019-01-04,1975947.0
2019-01-05,2229391.0
...,...
2022-12-27,NaN
2022-12-28,NaN
2022-12-29,NaN


### This travel dataset does not contain value for 16th May every year. So those 4 values are missing

In [56]:
df_combined.to_csv('daily_passenger_travel_data.csv', index=True)

### Combining X data with Covid cases and vaccines dataset

In [59]:
cleaned_cases_df = pd.read_csv('covid_cases_and_death_all_states_with_outlier_removal.csv')
cleaned_cases_df = cleaned_cases_df.sort_values(by=['date'])
cleaned_cases_df['Date'] = cleaned_cases_df['date']
cleaned_cases_df.drop(['date'], axis=1, inplace=True)
cleaned_cases_df.set_index('Date', inplace=True)
cleaned_cases_df

,state,total_cases_cum,total_death_cum,total_cases,total_death
Date,,,,,
2020-01-22,AK,0,0,0.0,0.0
2020-01-22,NJ,0,0,0.0,0.0
2020-01-22,NM,0,0,0.0,0.0
2020-01-22,NV,0,0,0.0,0.0
2020-01-22,NY,0,0,0.0,0.0
...,...,...,...,...,...
2022-05-13,NH,316691,2500,564.0,1.0
2022-05-13,RMI,17,0,0.0,0.0
2022-05-13,AL,1305124,19631,414.0,3.0


In [60]:
cleaned_cases_df = cleaned_cases_df.groupby('Date').sum()
# cleaned_cases_df.drop(['total_cases_cum', 'total_death_cum'], axis=1, inplace=True)
# cleaned_cases_df['Date'] = pd.to_datetime(cleaned_cases_df['Date'])
cleaned_cases_df.index = pd.to_datetime(cleaned_cases_df.index)
cleaned_cases_df

,total_cases_cum,total_death_cum,total_cases,total_death
Date,,,,
2020-01-22,0,0,0.0,0.0
2020-01-23,1,0,1.0,0.0
2020-01-24,2,0,1.0,0.0
2020-01-25,2,0,0.0,0.0
2020-01-26,3,0,1.0,0.0
...,...,...,...,...
2022-05-09,68868791,851512,47830.0,107.0
2022-05-10,69350764,873048,55556.0,223.0
2022-05-11,69333369,832177,67145.0,313.0


In [61]:
cleaned_vaccine_df = pd.read_csv('vaccination_all_states_cleaned.csv')
cleaned_vaccine_df['Date'] = pd.to_datetime(cleaned_vaccine_df['Date'])
cleaned_vaccine_df = cleaned_vaccine_df.sort_values(by=['Date'])
cleaned_vaccine_df.set_index('Date', inplace=True)
cleaned_vaccine_df.drop(['Unnamed: 0'], axis=1, inplace=True)
cleaned_vaccine_df

,Location,Distributed
Date,,
2020-12-13,LTC,0
2020-12-13,US,13650
2020-12-14,MT,1950
2020-12-14,NC,8775
2020-12-14,ND,3900
...,...,...
2022-05-14,MD,31800
2022-05-14,ME,6700
2022-05-14,MH,0


In [62]:
cleaned_vaccine_df = cleaned_vaccine_df.groupby('Date').sum()
cleaned_vaccine_df.index = pd.to_datetime(cleaned_vaccine_df.index)
cleaned_vaccine_df

,Distributed
Date,
2020-12-13,13650
2020-12-14,937950
2020-12-15,1870050
2020-12-16,369525
2020-12-17,2382900
...,...
2022-05-10,1198480
2022-05-11,1383400
2022-05-12,2580300


In [63]:
# df_combined

# merged_df = pd.merge(cleaned_cases_df, cleaned_vaccine_df, on="Date")
merged_df = pd.merge(cleaned_cases_df, df_combined, on="Date")
merged_df

,total_cases_cum,total_death_cum,total_cases,total_death,travel_count
Date,,,,,
2020-01-22,0,0,0.0,0.0,1645196.0
2020-01-23,1,0,1.0,0.0,2136584.0
2020-01-24,2,0,1.0,0.0,2004609.0
2020-01-25,2,0,0.0,0.0,1643435.0
2020-01-26,3,0,1.0,0.0,1777171.0
...,...,...,...,...,...
2022-05-09,68868791,851512,47830.0,107.0,2231804.0
2022-05-10,69350764,873048,55556.0,223.0,1900738.0
2022-05-11,69333369,832177,67145.0,313.0,2025161.0


In [64]:
merged_df.to_csv('cases_and_passenger_travel_data.csv', index=True)

### Uncleaned data

In [65]:
uncleaned_cases_df = pd.read_csv('covid_cases_and_death_all_states_without_outlier_removal.csv')
uncleaned_cases_df = uncleaned_cases_df.sort_values(by=['date'])
uncleaned_cases_df['Date'] = uncleaned_cases_df['date']
uncleaned_cases_df.drop(['date'], axis=1, inplace=True)
uncleaned_cases_df.set_index('Date', inplace=True)
uncleaned_cases_df

,state,total_cases_cum,total_death_cum,total_cases,total_death
Date,,,,,
2020-01-22,AK,0,0,0.0,0.0
2020-01-22,SC,0,0,0.0,0.0
2020-01-22,IN,0,0,0.0,0.0
2020-01-22,IL,0,0,0.0,0.0
2020-01-22,SD,0,0,0.0,0.0
...,...,...,...,...,...
2022-05-13,NE,482305,4234,0.0,0.0
2022-05-13,ND,242462,2274,146.0,2.0
2022-05-13,NC,2691805,24588,0.0,0.0


In [66]:
uncleaned_cases_df = uncleaned_cases_df.groupby('Date').sum()
uncleaned_cases_df.index = pd.to_datetime(uncleaned_cases_df.index)
uncleaned_cases_df.drop(['total_cases_cum', 'total_death_cum'], axis=1, inplace=True)
uncleaned_cases_df

,total_cases,total_death
Date,,
2020-01-22,0.0,0.0
2020-01-23,1.0,0.0
2020-01-24,1.0,0.0
2020-01-25,0.0,0.0
2020-01-26,1.0,0.0
...,...,...
2022-05-09,109390.0,168.0
2022-05-10,98285.0,365.0
2022-05-11,146393.0,553.0


In [67]:
uncleaned_vaccine_df = pd.read_csv('vaccination_all_states.csv')
uncleaned_vaccine_df['Date'] = pd.to_datetime(uncleaned_vaccine_df['Date'])
uncleaned_vaccine_df = uncleaned_vaccine_df.sort_values(by=['Date'])
uncleaned_vaccine_df.set_index('Date', inplace=True)
uncleaned_vaccine_df.drop(['Unnamed: 0'], axis=1, inplace=True)
uncleaned_vaccine_df

,Location,Distributed
Date,,
2020-12-13,AS,3900
2020-12-13,GU,3900
2020-12-13,LTC,0
2020-12-13,MP,4875
2020-12-13,US,13650
...,...,...
2022-05-14,MD,31800
2022-05-14,ME,6700
2022-05-14,MH,0


In [68]:
uncleaned_vaccine_df = uncleaned_vaccine_df.groupby('Date').sum()
uncleaned_vaccine_df.index = pd.to_datetime(uncleaned_vaccine_df.index)
uncleaned_vaccine_df

,Distributed
Date,
2020-12-13,27300
2020-12-14,937950
2020-12-15,1951950
2020-12-16,395850
2020-12-17,2382900
...,...
2022-05-10,1217980
2022-05-11,1383400
2022-05-12,2598300


In [69]:
# df_combined

# merged_clean_df = pd.merge(uncleaned_cases_df, uncleaned_vaccine_df, on="Date")
merged_clean_df = pd.merge(uncleaned_cases_df, df_combined, on="Date")
merged_clean_df

,total_cases,total_death,travel_count
Date,,,
2020-01-22,0.0,0.0,1645196.0
2020-01-23,1.0,0.0,2136584.0
2020-01-24,1.0,0.0,2004609.0
2020-01-25,0.0,0.0,1643435.0
2020-01-26,1.0,0.0,1777171.0
...,...,...,...
2022-05-09,109390.0,168.0,2231804.0
2022-05-10,98285.0,365.0,1900738.0
2022-05-11,146393.0,553.0,2025161.0


In [70]:
merged_clean_df.to_csv('clean_cases_and_passenger_travel_data.csv', index=True)